In [1]:
import sys

In [2]:
sys.path.remove('/home/jungeun/.local/lib/python3.6/site-packages')

In [3]:
import torch

In [4]:
sys.path.append('/home/jungeun/.local/lib/python3.6/site-packages')

In [5]:
import argparse
from torchsummary import summary
import tqdm
from tqdm import tqdm_notebook as tq
import os, time, math, copy,random
import numpy as np
import torch.nn as nn
import torchvision
#from torchvision import transforms, datasets
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from collections import namedtuple
import matplotlib.pyplot as plt
import datetime

torch.set_printoptions(precision=8, linewidth=50000)
import warnings
warnings.filterwarnings(action='ignore')


In [6]:
BLACK	= '\033[30m'
RED		= '\033[31m'
GREEN	= '\033[32m'
YELLOW	= '\033[33m'
BLUE	= '\033[34m'
MAGENTA	= '\033[35m'
CYAN	= '\033[36m'
RESET	= '\033[0m'
SEL		= '\033[7m'

In [7]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

In [8]:
def XOR(iA,iB):
    if iA != iB :
        iOut = 1
    else : 
        iOut = 0
    return iOut

In [9]:
def snum(a):
    if a >= 0 :
        return 0
    else :
        return 1

In [10]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

In [11]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        if int(self.b0) + int(self.b1) + int(self.b2) + int(self.b3) + int(self.b4) + int(self.b5) + int(self.b6) == 0 :
            self.b0 = eval(f'str(random.randint(0,1))')
            self.b1 = eval(f'str(random.randint(0,1))')
            self.b2 = eval(f'str(random.randint(0,1))')
            self.b3 = eval(f'str(random.randint(0,1))')
            self.b4 = eval(f'str(random.randint(0,1))')
            self.b5 = eval(f'str(random.randint(0,1))')
            self.b6 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = str(XOR(int(stream[5]),int(stream[6])))
        self.b1 = str(stream[0])
        self.b2 = str(stream[1])
        self.b3 = str(stream[2])
        self.b4 = str(stream[3])
        self.b5 = str(stream[4])
        self.b6 = str(stream[5])
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

In [12]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

In [13]:
def Comp(a,lfsr,snum): #lfsr number < origin number , output = 1
    for com in range(0,len(a)):
        oA = 0
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = 1
            else :
                oA = 0
            break
    return XOR(oA,snum)

In [14]:
def perm(a):
    al = len(a)
    blist = []
    if args.frac_bits == args.bBW : 
        for i in range(al) :
            blist.append(a[al-i-1])
    elif args.frac_bits > args.bBW :
        for i in range(al-(args.frac_bits-args.bBW)) :
            blist.append((a[al-(args.frac_bits-args.bBW)-i-1]))
    b = "".join(blist)
    b = b + ('0'*(args.frac_bits-args.bBW))
    return b

In [15]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

In [16]:
def CountOne(nIn):
    nlist = []
    for num in nIn.view(-1,nIn.size()[-1]):
        n = 0
        for a in num:
            if a == 1 :
                n += 1
        if num[0] == 1 :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.size()[0],nIn.size()[1])

In [17]:
def CountOne1(nIn):
    n = 0
    for num in nIn :
        if num.item() == 1 :
            n += 1
    if nIn[0].item() == 1 :
        n = n - 1
    return n

In [18]:
def defSign(nIn):
    nlist = []
    k = nIn.view(-1,nIn.size()[-1])
    for num in nIn.view(-1,nIn.size()[-1]):
        if num[0] == 1 :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.size()[0],nIn.size()[1])

In [19]:
def defSign1(nIn):
    if nIn[0].item() == 1 :
        return -1
    else :
        return 1

In [20]:
def SNG(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    #sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return torch.zeros((2**args.bBW)+1)
    else :
        return torch.tensor(oAlist)

In [21]:
def SNGtensor(fIn,lfsr):
    sList = []
    fsize = list(fIn.size())
    fsize.insert(2,(2**args.bBW)+1)
    for aTensor in fIn.view(-1):
        sList.append(SNG(aTensor,lfsr))
    
    c = torch.stack(sList,0)
    return c.view(fsize)

In [22]:
def SNG_P(iIN,lfsr):
    if iIN == 1:
        iIN = 0.9999
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = perm(lfsr[k])
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    #sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return torch.zeros((2**args.bBW)+1)
    else :
        return torch.tensor(oAlist)

In [23]:
def SNGPtensor(fIn,lfsr):
    sList = []
    fsize = list(fIn.size())
    fsize.insert(2,(2**args.bBW)+1)
    for aTensor in fIn.view(-1):
        sList.append(SNG_P(aTensor,lfsr))
    
    c = torch.stack(sList,0)
    return c.view(fsize)

In [24]:
def mul(a,b):
    outlist = []
    
    if len(a) != len(b) :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for anum,bnum in zip(a[1:],b[1:]) :
        try :
            outlist.append(int(anum)&int(bnum))
        except RuntimeError :
            print(anum.type())
            print(bnum.type())
            
    o = torch.tensor(outlist)
    return o 

In [25]:
def Multensor(aIn,wIn):
    olist = []
    for a,w in zip(aIn.view(-1,aIn.size()[-1]),wIn.view(-1,wIn.size()[-1])):
        o = mul(a,w)
        olist.append(o)
    out = torch.stack(olist,0)
    return out.view(aIn.size()[0],wIn.size()[1],-1)

In [26]:
def Convert(sIn,SF):
    s = defSign(sIn)
    o = (CountOne(sIn)/(2**args.bBW))*SF*s
    return o

In [27]:
def SCmulTensor(aIn,wIn,aSF,wSF):
    a = aIn/aSF
    w = wIn/wSF
    
    lfsr = LFSRlist7()
    
    Sa = SNGtensor(a,lfsr)
    Sw = SNGPtensor(w,lfsr)
    
    Smul = Multensor(Sa,Sw)
    
    Bout = Convert(Smul,aSF*wSF)
    
    return Bout

In [28]:
def StoConv(X, filters, bias, stride=1, pad=0):
    
    n, c, h, w = X.shape # 1, 1, 32, 32
    n_f, _, filter_h, filter_w = filters.shape

    out_h = (h+2*pad-filter_h)//stride + 1
    out_w = (w+2*pad-filter_w)//stride + 1
    # add padding to height and width.
    in_X = F.pad(X,(0,0,0,0,pad,pad,pad,pad),"constant", 0)
    out  = torch.zeros((n, n_f, out_h, out_w))
    
    for i in range(n): # for each image.
        aSF = findMaxMin(in_X[i])
        for c in range(n_f): # for each channel.
            wSF = findMaxMin(filters[c])
            for h in range(out_h): # slide the filter vertically.
                h_start = h * stride
                h_end = h_start + filter_h
                for w in range(out_w): # slide the filter horizontally.
                    w_start = w * stride
                    w_end = w_start + filter_w
                    # Element-wise multiplication.
                    out[i, c, h, w] = torch.sum(SCmulTensor(in_X[i,:,h_start:h_end,w_start:w_end],filters[c],aSF,wSF))
    
    return out

In [29]:
class	fxp:
	def	__init__(self, bIn, iBWF):
		self.iFullBW	= len(bIn)
		self.iIntgBW	= self.iFullBW - iBWF
		self.bSign		= bIn[0]
		self.bIntg		= bIn[:self.iIntgBW]
		self.bFrac		= bIn[self.iIntgBW:]
		self.fFull		= 0
		try:
			for idx, bit in enumerate(bIn):
				if	idx == 0:
					self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
				else:
					self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
		except:
			print(bIn)
		self.dispFull	= RED + self.bIntg + BLUE + self.bFrac + RESET
		return

In [30]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        #if fIn < self.fMin or fIn > self.fMax:
            #print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [31]:
def	flp2fixTensor(fIn, iBW, iBWF):
	fMin = - 2 ** (iBW - iBWF - 1)
	fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
	fList = []
	for aTensor in fIn.view(-1):
		fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
	return torch.tensor(fList).view(fIn.size())

In [32]:
parser = argparse.ArgumentParser(description='PyTorch for MNIST dataset')
parser.add_argument('--device', type=str, default='cuda', help='Device')
#parser.add_argument('--shuffle', action='store_true', default=False, help='enables data shuffle')
#parser.add_argument('--dataset', type=str, default='mnist', help='training dataset')
#parser.add_argument('--data_path', type=str, default=data_path, help='path to MNIST')
parser.add_argument('--batch_size', type=int, default=64, help='batch size')
parser.add_argument('--epochs', type=int, default=30, help='number of epochs to train')
parser.add_argument('--lr', type=float, default=0.001, help='learning rate')
parser.add_argument('--optimizer', type=str, default='adam', help='optimizer')
parser.add_argument('--loss_func', type=str, default='cel', help='optimizer')
parser.add_argument('--quant_opt', type=str, default='asym', help='Type of Quantization')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--pretrained', type=bool, default=True, help='Pretrained Model')
parser.add_argument('--act_quant', type=bool, default=False, help='Activation Quantization')
parser.add_argument('--disp', type=bool, default=False, help='Display Model Information')
parser.add_argument('--bBW',type=int,default=7,help='bit number')
args = parser.parse_args(args=[])


In [33]:
X = torch.tensor([
# image 1
[
    [[1, 2, 9, 2, 7],
    [5, 0, 3, 1, 8],
    [4, 1, 3, 0, 6],
    [2, 5, 2, 9, 5],
    [6, 5, 1, 3, 2]],

    [[4, 5, 7, 0, 8],
    [5, 8, 5, 3, 5],
    [4, 2, 1, 6, 5],
    [7, 3, 2, 1, 0],
    [6, 1, 2, 2, 6]],

    [[3, 7, 4, 5, 0],
    [5, 4, 6, 8, 9],
    [6, 1, 9, 1, 6],
    [9, 3, 0, 2, 4],
    [1, 2, 5, 5, 2]]
],
# image 2
[
    [[7, 2, 1, 4, 2],
    [5, 4, 6, 5, 0],
    [1, 2, 4, 2, 8],
    [5, 9, 0, 5, 1],
    [7, 6, 2, 4, 6]],

    [[5, 4, 2, 5, 7],
    [6, 1, 4, 0, 5],
    [8, 9, 4, 7, 6],
    [4, 5, 5, 6, 7],
    [1, 2, 7, 4, 1]],

    [[7, 4, 8, 9, 7],
    [5, 5, 8, 1, 4],
    [3, 2, 2, 5, 2],
    [1, 0, 3, 7, 6],
    [4, 5, 4, 5, 5]]
]
])
print('Images:', X.size())

filters = torch.tensor([
# kernel 1
[
    [[1, 0, 1],
    [0, 1, 0],
    [1, 0, 1]],

    [[3, 1, 3],
    [1, 3, 1],
    [3, 1, 3]],

    [[1, 2, 1],
    [2, 2, 2],
    [1, 2, 1]]
],
# kernel 2
[
    [[5, 1, 5],
    [2, 1, 2],
    [5, 1, 5]],

    [[1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]],

    [[2, 0, 2],
    [0, 2, 0],
    [2, 0, 2]],
],
# kernel 3
[
    [[5, 1, 5],
    [2, 1, 2],
    [5, 1, 5]],

    [[1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]],

    [[2, 0, 2],
    [0, 2, 0],
    [2, 0, 2]],
]
])
print('Filters:', filters.size())

Images: torch.Size([2, 3, 5, 5])
Filters: torch.Size([3, 3, 3, 3])


In [35]:
out = StoConv(X, filters, 0,stride=2, pad=0).to(args.device)
print('Output:', out.shape)
print(out)

Output: torch.Size([2, 3, 2, 2])
tensor([[[[222.53906250, 249.75000000],
          [201.23437500, 207.56250000]],

         [[330.46875000, 358.59375000],
          [299.53125000, 276.32812500]],

         [[331.17187500, 361.75781250],
          [297.07031250, 288.28125000]]],


        [[[242.15625000, 232.45312500],
          [225.49218750, 235.82812500]],

         [[350.50781250, 342.07031250],
          [304.80468750, 330.82031250]],

         [[339.60937500, 333.63281250],
          [309.72656250, 332.57812500]]]], device='cuda:0')
